In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import os
import glob
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline


from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.utils import *
from keras.initializers import *
import tensorflow as tf
import time, random
import re
import string

from string import digits
from keras.optimizers import Adam



from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

from collections import Counter
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

Using TensorFlow backend.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
c:\users\user\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:44: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [2]:
import os
short_files = []
cwd = os.getcwd()

#name of the folder having data set in the directory
path = cwd+'\\short_stories'
os.listdir() 

print(path)

# get data file names
filenames = glob.glob(path + "\\*.story")

for file in filenames:
    with open(file, encoding="utf8") as f1:
        short_files.append(f1.read())

C:\Users\user\siddhartha_ireddy\cnn_dm\short_stories


In [3]:
len(short_files)

12500

In [4]:
def sent_split(a):
  corrected = str(a)
  corrected = re.sub(r"\b(CNN)\b",r"", corrected)
  corrected = re.sub(r'(["!?;])\1+', r'\1', corrected)
  corrected = re.sub(r'\{2,}', r'...', corrected)
  corrected = re.sub(r"//t",r"\t", corrected)
  corrected = re.sub(r"( )\1+",r"\1", corrected)
  corrected = re.sub(r"(\n)\1+",r"\1", corrected)
  corrected = re.sub(r"(\r)\1+",r"\1", corrected)
  corrected = re.sub(r"(\t)\1+",r"\1", corrected)
  corrected.translate(str.maketrans('', '', string.punctuation))
  corrected = corrected.split("highlight",maxsplit=1)
  return corrected

def sent_clean(b):
  b = b.split("\n")
  abc = [] 
  for j in b:
    tokenizer = RegexpTokenizer(r'\w+')
    j = " ".join(tokenizer.tokenize(j))
    k_1 = []
    if len(j)>1:
      p = word_tokenize(j)
      for i in p:
        if i.lower() not in stop_words:
          k_1.append(i.lower())
    if len(k_1) > 0:
      abc.append(" ".join(k_1))
  return " ".join(abc)


def highlight_clean(b):
  abc = [] 
  
  b = re.sub(r"\bhighlight\b",r"", b)
  b = b.split("\n")
  k_1 = []
  for j in b:
    tokenizer = RegexpTokenizer(r'\w+')
    j = " ".join(tokenizer.tokenize(j))
    k_1 = []
    if len(j)>1:
      p = word_tokenize(j)
      for i in p:
        if i.lower() not in stop_words:
          k_1.append(i.lower())
    if len(k_1) > 0:
      abc.append(" ".join(k_1))
  return " ".join(abc)

In [5]:
stop_words = set(stopwords.words('english'))

In [6]:
data_set = pd.DataFrame(columns=['sentence', 'highlight'])
data_set_clean = pd.DataFrame(columns=['cleaned sentence', 'cleaned highlight'])

In [ ]:
for i in range(len(short_files)):
  data_set.loc[i,'sentence'] = sent_split(short_files[i])[0]
  data_set.loc[i,'highlight'] = sent_split(short_files[i])[1]

for i in range(len(short_files)):
  data_set_clean.loc[i,'cleaned sentence'] = sent_clean(sent_split(short_files[i])[0])
  data_set_clean.loc[i,'cleaned highlight'] = highlight_clean(sent_split(short_files[i])[1])

In [ ]:
data_set_clean.head()

In [10]:
data_set_clean.isnull().sum()

cleaned sentence     0
cleaned highlight    0
dtype: int64

In [11]:
data_set_clean.drop_duplicates(inplace=True)

In [12]:
# Lowercase all characters
data_set_clean['cleaned sentence'] = data_set_clean['cleaned sentence'].apply(lambda x: x.lower())
data_set_clean['cleaned highlight'] = data_set_clean['cleaned highlight'].apply(lambda x: x.lower())

In [13]:
# Remove quotes
data_set_clean['cleaned sentence'] =data_set_clean['cleaned sentence'].apply(lambda x: re.sub("'", '', x))
data_set_clean['cleaned highlight'] =data_set_clean['cleaned highlight'].apply(lambda x: re.sub("'", '', x))

In [14]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
data_set_clean['cleaned sentence'] =data_set_clean['cleaned sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
data_set_clean['cleaned highlight']=data_set_clean['cleaned highlight'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [15]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
data_set_clean['cleaned sentence']=data_set_clean['cleaned sentence'].apply(lambda x: x.translate(remove_digits))
data_set_clean['cleaned highlight']=data_set_clean['cleaned highlight'].apply(lambda x: x.translate(remove_digits))
data_set_clean['cleaned highlight'] = data_set_clean['cleaned highlight'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
data_set_clean['cleaned highlight']=data_set_clean['cleaned highlight'].apply(lambda x: x.strip())
data_set_clean['cleaned sentence']=data_set_clean['cleaned sentence'].apply(lambda x: x.strip())
data_set_clean['cleaned highlight']=data_set_clean['cleaned highlight'].apply(lambda x: re.sub(" +", " ", x))
data_set_clean['cleaned sentence']=data_set_clean['cleaned sentence'].apply(lambda x: re.sub(" +", " ", x))


In [16]:
data_set_clean.head()

cleaned sentence  \
0  official u president barack obama wants lawmakers weigh whether use military force syria obama sent letter heads house senate saturday night hours announcing believes military action syrian targets right step take alleged use chemical weapons proposed legislation obama asks congress approve use military force deter disrupt prevent degrade potential future uses chemical weapons weapons mass destruction step set turn international crisis fierce domestic political battle key questions looming debate u n weapons inspectors find syria happens congress votes syrian government react televised address white house rose garden earlier saturday president said would take case congress wants believe authority carry military action without specific congressional authorization know country stronger take course actions even effective said debate issues big business usual obama said top congressional leaders agreed schedule debate body returns washington september senate foreign relations committee hold hearing matter tuesday sen robert menendez said transcript read obama full remarks syrian crisis latest developments u n inspectors leave syria obama remarks came shortly u n inspectors left syria carrying evidence determine whether chemical weapons used attack early last week damascus suburb aim game mandate clear ascertain whether chemical weapons used u n spokesman martin nesirky told reporters saturday used weapons reported toxic gas attack damascus suburb august key point global debate syrian crisis top u officials said doubt syrian government behind syrian officials denied responsibility blamed jihadists fighting rebels british u intelligence reports say attack involved chemical weapons u n officials stressed importance waiting official report inspectors inspectors share findings u n secretary general ban ki moon ban said wants wait u n team final report completed presenting u n security council organization prohibition chemical weapons nine inspectors belong said saturday could take three weeks analyze evidence collected needs time able analyze information samples nesirky said noted ban repeatedly said alternative political solution crisis syria military solution option bergen syria problem hell u obama menace must confronted obama senior advisers debated next steps take president comments saturday came amid mounting political pressure situation syria u lawmakers called immediate action others warn stepping could become quagmire global leaders expressed support british parliament vote military action earlier week blow obama hopes getting strong backing key nato allies saturday obama proposed said would limited military action syrian president bashar al assad military attack would open ended include u ground forces said syria alleged use chemical weapons earlier month assault human dignity president said failure respond force obama argued could lead escalating use chemical weapons proliferation terrorist groups would people harm world many dangers menace must confronted syria missile strike would happen next map u allied assets around syria obama decision came friday night friday night president made last minute decision consult lawmakers happen vote unclear senior administration official told obama authority act without congress even congress rejects request authorization use force obama saturday continued shore support strike al assad government spoke phone french president francois hollande rose garden speech two leaders agreed international community must deliver resolute message assad regime others would consider using chemical weapons crimes unacceptable violate international norm held accountable world white house said meanwhile uncertainty loomed congress would weigh u military officials said remained ready key assertions u intelligence report syria syria wants chemical weapons horror reactions mixed obama speech spokesman syrian national coalition said opposition group disappointed obama announcement fear l

In [17]:
for i in range(len(data_set_clean)):
  if len(data_set_clean.iloc[i, 0]) >= 200:
    data_set_clean.iloc[i, 0] = data_set_clean.iloc[i, 0][:200]
  
  if len(data_set_clean.iloc[i, 1]) >= 200:
    data_set_clean.iloc[i, 1] = data_set_clean.iloc[i, 1][:200]


In [18]:
data_set_clean.head()

,cleaned sentence,cleaned highlight
0,official u president barack obama wants lawmakers weigh whether use military force syria obama sent letter heads house senate saturday night hours announcing believes military action syrian targets ri,syrian official obama climbed top tree know get obama sends letter heads house senate obama seek congressional approval military action syria aim determine whether cw used says u n spokesman
1,los angeles medical doctor vancouver british columbia said thursday california arson suspect harry burkhart suffered severe mental illness examined part team doctors dr blaga stancheva family physicia,new canadian doctor says part team examining harry burkhart new diagnosis autism severe anxiety post traumatic stress disorder depression burkhart also suspected german arson probe officials say prose
2,police arrested another teen thursday sixth suspect jailed connection gang rape year old girl northern california high school campus jose carlos montano arrested charges felony rape rape concert force,another arrest made gang rape outside california school investigators say people took part stood watched assault four suspects appeared court thursday three wore bulletproof vests
3,four groups advocate immigrant rights said thursday challenge arizona new immigration law allows police ask anyone proof legal u residency mexican american legal defense educational fund american civi,new groups announce legal challenge phoenix american civil liberties union aclu arizona national immigration law center slam law mexican american legal defense educational fund also objects say law en
4,second time papacy pope francis announced new group bishops archbishops set become cardinals come world pope francis said sunday would hold meeting cardinals february name new cardinals coming countri,new cardinals installed february come countries myanmar tonga americans made list time previous time francis papacy


In [19]:
data_set_clean.reset_index(inplace = True)

In [20]:
data_set_clean.columns

Index(['index', 'cleaned sentence', 'cleaned highlight'], dtype='object')

In [21]:
data_set_clean.drop(labels = ['index'], inplace = True, axis = 1)

In [22]:
#Hyperparameters
batch_size = 64
latent_dim = 256
num_samples = 5000

In [23]:
data_set_clean.columns

Index(['cleaned sentence', 'cleaned highlight'], dtype='object')

In [24]:
#Vectorize the data.
input_texts = []
target_texts = []
input_chars = set()
target_chars = set()


    
for i in range(min(num_samples, len(data_set_clean) - 1)):
    input_text = data_set_clean.loc[i,'cleaned sentence']
    target_text = data_set_clean.loc[i,'cleaned highlight']
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    for char in input_text:
        if char not in input_chars:
            input_chars.add(char)
    for char in target_text:
        if char not in target_chars:
            target_chars.add(char)

input_chars = sorted(list(input_chars))
target_chars = sorted(list(target_chars))
num_encoder_tokens = len(input_chars)
num_decoder_tokens = len(target_chars)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

#Print size
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 5000
Number of unique input tokens: 37
Number of unique output tokens: 41
Max sequence length for inputs: 200
Max sequence length for outputs: 202


In [25]:
#Define data for encoder and decoder
input_token_id = dict([(char, i) for i, char in enumerate(input_chars)])
target_token_id = dict([(char, i) for i, char in enumerate(target_chars)])

encoder_in_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')

decoder_in_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_in_data[i, t, input_token_id[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_in_data[i, t, target_token_id[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_id[char]] = 1.

In [32]:
#Define and process the input sequence
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
#We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

#Using `encoder_states` set up the decoder as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [33]:
#Final model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [34]:
#Model Summary
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 37)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 41)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 301056      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  305152      input_2[0][0]                    
                                                                 lstm_1[0][1]               

In [35]:
#Model data Shape
print("encoder_in_data shape:",encoder_in_data.shape)
print("decoder_in_data shape:",decoder_in_data.shape)
print("decoder_target_data shape:",decoder_target_data.shape)

encoder_in_data shape: (5000, 200, 37)
decoder_in_data shape: (5000, 202, 41)
decoder_target_data shape: (5000, 202, 41)


In [36]:
#Compiling and training the model
model.compile(optimizer=Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.001), loss='categorical_crossentropy')

model.fit([encoder_in_data, decoder_in_data], decoder_target_data, batch_size = batch_size, epochs=10, validation_split=0.2)

Train on 4000 samples, validate on 1000 samples
Epoch 1/10
4000/4000 [==============================] - 792s 198ms/step - loss: 2.6008 - val_loss: 2.2529
Epoch 2/10
4000/4000 [==============================] - 773s 193ms/step - loss: 2.1619 - val_loss: 2.1102
Epoch 3/10
4000/4000 [==============================] - 770s 193ms/step - loss: 2.0804 - val_loss: 2.0623
Epoch 4/10
3520/4000 [=========================>....] - ETA: 1:26 - loss: 2.0395

KeyboardInterrupt: 

In [ ]:
#Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [ ]:
reverse_input_char_index = dict((i, char) for char, i in input_token_id.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_id.items())

#Define Decode Sequence
def decode_sequence(input_seq):
    #Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    #Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    #Get the first character of target sequence with the start character.
    target_seq[0, 0, target_token_id['\t']] = 1.

    #Sampling loop for a batch of sequences
    #(to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        #Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        #Exit condition: either hit max length
        #or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        #Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        #Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
for seq_index in range(20):
    input_seq = encoder_in_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)